In [ ]:
!pip install diffusers transformers torch torchvision accelerate opencv-python scikit-image customtkinter

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from skimage.metrics import structural_similarity as ssim
import numpy as np
import cv2
import customtkinter as ctk
from PIL import Image


pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipe = pipe.to("cuda")

def generate_image(prompt, output_path="generated_image.png"):
    """
    Generate an image from a text prompt using Stable Diffusion.
    """
    with torch.autocast("cuda"):
        image = pipe(prompt).images[0]
    image.save(output_path)
    return image

def compute_ssim(image1_path, image2_path):
    """
    Compute SSIM (Structural Similarity Index) between two images.
    """
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)
    score, _ = ssim(img1, img2, full=True)
    return score

# GUI with customtkinter
def start_gui():
    def on_generate():
        prompt = prompt_entry.get()
        image = generate_image(prompt)
        output_label.configure(text="Image generated successfully!")
        img_display = Image.open("generated_image.png")
        img_display.thumbnail((300, 300))
        img_display = ctk.CTkImage(img_display)
        image_label.configure(image=img_display)
        image_label.image = img_display

    root = ctk.CTk()
    root.geometry("500x600")
    root.title("Text-to-Image Generator")

    prompt_label = ctk.CTkLabel(root, text="Enter a prompt:")
    prompt_label.pack(pady=10)

    prompt_entry = ctk.CTkEntry(root, width=400)
    prompt_entry.pack(pady=10)

    generate_button = ctk.CTkButton(root, text="Generate Image", command=on_generate)
    generate_button.pack(pady=20)

    output_label = ctk.CTkLabel(root, text="")
    output_label.pack(pady=10)

    image_label = ctk.CTkLabel(root, text="")
    image_label.pack(pady=10)

    root.mainloop()

if __name__ == "__main__":
    start_gui()
